<div style='background-image: url("../../resources/section_header.jpg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: x-large ; font-weight: 900 ; color: rgba(0 , 115 , 207 , 0.9) ; line-height: 100%">Python for Computational Linguists</div>
            <div style="font-size: x-large ; padding-top: 20px ; color: rgba(0 , 115 , 207, 0.7)">2.3 Part-of-speech Tagging</div>
        </div>
    </div>
</div>

# Module 2.3

We agreed to be flexible on the content and just to include as much as could comfortably fit into 2 hours with the potential for extensional activities.
 material from http://www.datascienceassn.org/sites/default/files/Natural%20Language%20Processing%20with%20Python.pdf
 data from
http://alias-i.com/lingpipe/demos/tutorial/posTags/read-me.html

* Summary of key objectives
** understand that many language processing tasks, such as part of speech tagging, can be viewed as text classification;
** understand that part of speech tagging is an important early application of sequence classification;
** see in practice how a simple n-gram model can be used to perform part of speech tagging;
** see in practice how linguistic features found in corpora can help language models to understand linguistic patterns, and be used to make predictions about new language data;

* Quick introduction (with material taken from the NLTK book and the lecture notes):
** What is part of speech tagging?
** What is a tagged corpus?
** Using spaCy’s part of speech tagger to process a sequence of words
Quick reminder of how to use spaCy’s POS tagger and some of the issues we raised, e.g. about domain adaptation, data quantity and quality. E.g. get the students to POS tag two contrasting sentences, e.g. lexically ambiguous homophones - one with a common homonym sense and another with a rare homonym sense (e.g. ‘It is wrong to object to this object’ or ‘I must present the present on his birthday’, ‘The insurance for the invalid was invalid’), or one sentence with a neologism and one without a neologism or one with a nonsense word/unknown word that is not a common noun (e.g. ‘he was scrobbling’). Encourage the students to play with the tagger by submitting their own challenging sentences.  Get the students to notice the set of POS tags and their meanings.  Get the students to see what happens if you normalise the text to lower case. 
In Module 1 we saw how to call spaCy’s POS tagger using a function call.  Whilst spaCy’s POS tagger is very good, we’re going to be going hands on by building our own POS simple n-gram tagger using the NLTK library.
* Today’s goal: building and testing a part of speech tagger with NLTK
* Getting started
** Exploring tagged corpora (section 5.2 in the NLTK book)
Look at several POS tagged corpora, e.g. the Brown corpus and the GENIA corpus (see data at http://alias-i.com/lingpipe/demos/tutorial/posTags/read-me.html). 
Try to get the students thinking about the POS tag set – what do the categories mean?  Get the students to explore the corpora and gain insights, e.g. most frequent POS tag, least frequent POS tag, plotting the curve of the POS tag frequency distribution.  What kinds of words occur in the noun, adjective, adverb category?  
Look at some n-gram sequences of POS tags to get the students thinking about n-grams.
Look at some words which are POS ambiguous.
** Mapping words to properties using Python dictionaries (section 5.3 in NLTK)
Get the students thinking about linguistic objects as data structures, e.g. a dictionary could map from a headword to a POS or to a specific sense, e.g. in WordNet or Wikipedia. 
Dictionaries in Python again. Defining a POS dictionary by hand.  Creating a POS dictionary using a tagged corpus. 
** Creating a bigram dictionary
Page 196 in the NLTK book shows how to create a dictionary of words and tags to perform bigram tagging;
If we try to create an n-gram tagger for larger values of n what will happen?  We’ll encounter the sparse data problem and also start to reach the limits of computer memory.
* Automatic tagging (section 5.4 in the NLTK book)
** Separating training and testing data
From page 225 in the NLTK book – uses the Brown corpus to make a 90:10 split
Get the students to fill in a LaTeX table that characterises the data: distribution of tags in the whole corpus, in the 90% training set and the 10% testing set.
** A simple default tagger 
Assigns the NN to each word in the Brown training corpus
Evaluate it using the Brown testing corpus with recall, precision and F1 for each category
Get the students to measure the out of vocabulary rate between the training and test data sets
-	Ask the students to report the number of types and tokens in the 90% training set, the 10% testing set and the OOV rate.
Get the students to look and learn from the evaluation data
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
** A regular expression tagger  cut out
Again, evaluate it using recall, precision and F1 for each category 
Again encourage the students to look and learn from the evaluation data
Get the students to construct new regular expression patterns and to evaluate them
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
** The most frequent 100 unigram tagger
Again, evaluate it using recall, precision and F1 for each category 
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
Again encourage the students to look and learn from the evaluation data
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
** A simple bi-gram tagger
Emphasize that a real n-gram tagger would do decoding using a dynamic programming algorithm called Viterbi (the students will have heard of this in the lecture).  Today in the interests of time we are not going to do decoding, we’re just going to assign the most frequent POS sequence to each bigram that matches (if indeed a bigram does match).
Again, evaluate it using recall, precision and F1 for each category 
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
Low accuracy because of data sparsity – most bigrams were never seen during training.
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
** Combining a models with backoff (e.g. unigram plus bigram tagger)
** Further insights from evaluation using a confusion matrix
* Reminder of key objectives
** understand that many language processing tasks, such as part of speech tagging, can be viewed as text classification;
** understand what a part of speech tag set is;
** see in practice how a simple n-gram model can be used to perform part of speech tagging;
** see in practice how linguistic features found in corpora can help language models to understand linguistic patterns, and be used to make predictions about new language data;

* Homework: 
** (1 point): Run your bigram tagger on this made-up sentence ‘@Will WOOOHOO Will the New jPhone bOut 2night? Soexcited :)‘. Does the tagger get the correct answer?  Write down two challenges the tagger has incorrectly resolved.  looking here for an OOV word (WOOOHOOO) and  failure to disambiguate homonyms (e.g. Will).  The tagger may also not handle slang (2night), neologisms (jPhone) and tokenization errors (Soexcited).
** (2 points): Write a program to find out:
a.	which word has the most POS tags in the GENIA corpus.
b.	what percentage of words in the GENIA corpus are ambiguous, i.e. have more than one POS tag.
** (2 points):  Write a program to replace low frequency words (words with a frequency of 5 or less) with UNK in the Brown corpus.  How much does this improve your backoff tagger’s F-score performance?

This module is built based on the lecture note about part-of-speech tagging and Chapter 5 of the [book](http://www.datascienceassn.org/sites/default/files/Natural%20Language%20Processing%20with%20Python.pdf), and students are strongly recommended to go through them.

In this module, we are going to
- a
- b
- c


# Python for Computational Linguists 1.4: Part-of-speech Tagging (POS Tagging)

### What is POS tag?
For each word in context, we can assign a **lexical category**, such as noun, verbs, adjectives, etc.
These categories are often referred to as a word's part-of-speech tag or POS tag.
The set of all POS tags is called a **tagset**, and the activity of assigning these tags to words is referred to as **part-of-speech tagging** or simply as **POS tagging**.

The number of categories we require for our tagset often depends on both linguistic and practical considerations.
Very commonly used tagsets include the 87-tag Brown set, 45-tag Penn Treebank set, the 61-tag CLAWS 5 (C5), and the 17-tag Universal POS tagset. 
[Here](https://universaldependencies.org/u/pos/) is the list of tags of Universal POS tagset.
Please check lecture note for more detials.

### Why do we want to do POS tagging?
As the POS tag of a word will give a large amount of information about this word and its neighbours, POS tagging can help understanding better texts, beneficial to many dowstream NLP tasks such as:

1. Disambiguating word senses. Consider the following example for the heterophone *content*:
    1. There was very little *content* to the essay.
    2. The sleepy pug puppy was very *content*.
  
  or homonyms:
  - It is wrong to *object* to this *object*.
  - I must *present* the *present* on his birthday.
  - The insurance for the *invalid* was *invalid*.

2. Mitigating the issue of data sparsity. 
    - Label named entities like people, places or organizations as part of information extraction systems. In the following example, both *Chase Manhattan* and *J.P. Morgan* should be detected as proper nouns:
        - Chase Manhattan and its merger partner J.P. Morgan.
    
    - Deal with out-of-vocabulary (OOV) words such as neologism words or acronyms in the social media: 
        - @username its #awesome u gonna ♥ it Chk out our cooool project on some_url + RT it.


More generally, being able to automatically perform POS tagging will help reduce the laborious human effort required to parse a sentence, and it will be the main goal of this module, which is to build such an automatic model/tagger.

## ❓ Pre-module quiz

TODO: any quiz more interactive????

## Getting Started

Before building our own tagger, we are going to use the NLTK library to load some existing tagger to perform POS tagging, and some tagged corpora will be introduced for further training and evaluations of our models.
First let's import the [NLTK](https://www.nltk.org/) library and download some necessary resources.

In [ ]:
import nltk
nltk.__version__
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('treebank')
nltk.download('tagsets')
nltk.download('gutenberg')

Given the input text, we first tokenize it.

In [ ]:
text = 'I like this module and NLTK library very very much!'
tokenized_text = nltk.word_tokenize(text)
print(tokenized_text)

We can see that the tokenized text is a list where each element is a token. 

Now let's tag the tokenize text.

In [ ]:
tagged_text = nltk.pos_tag(tokenized_text)
print(tagged_text)

The returned tagged text is a list, where each element is a tuple. For each tuple, the first element is the previous token, and the second element is its corresponding tag.

NLTK provides documentation for each tag, which can be queried using the tag, e.g., `nltk.help.upenn_tagset('PRP')`, or a regular expression, e.g., `nltk.help.upenn_brown_tagset('NN.*')`. Some corpora have README files with tagset documentation; see `nltk.name.readme()`, substituting in the name of the corpus.

> **<h3>💻 Try it yourself!</h3>**

Use the current tagger to tag the previous text examples to see if the key words (heterophone, homonym, named entity and neologism) are correctly tagged.
What are possible explanations to the wrong words?
What happens if we normalise the text to lower case?
Play with the tagger by submitting your own challenging sentences.

- There was very little *content* to the essay.
- The sleepy pug puppy was very *content*.
- It is wrong to *object* to this *object*.
- I must *present* the *present* on his birthday.
- The insurance for the *invalid* was *invalid*.
- They *refuse* to permit us to obtain the *refuse* permit
- Chase Manhattan and its merger partner J.P. Morgan.
- @username its #awesome u gonna ♥ it Chk out our cooool project on some_url + RT it.

TODO: have answers ready and ask students to identify any wrong ones, and ask them why? 
change the sentences to let the tagger change the decision wrong -> correct, correct -> wrong

### Connection with Text Classification

As we have seen in previous modules, text classification is the task where we assign a label to the whole sequence of text, e.g. document. POS tagging can be also viewd similarly. Instead of assigning a label on the document level, we assign a label to each word or token. This type of task is usually called **sequence labeling** or **sequence classification**, and POS tagging is one of the most important tasks of sequence classification in NLP.

### Tagged corpora
POS taggers using machine learning techniques generally require annotated data to train on.
For all tagging models, including those that do not require any training data like regex- or rule-based models, an annotated evaluation dataset is also required to measure the performance of these models by comparing the model outputs with the gold standard annotations.

Tagged corpora, generally tagged by an human expert, have the form similar to the following:

```
The/AT grand/JJ jury/NN commented/VBD on/IN a/AT number/NN of/IN other/AP topics/NNS ,/, AMONG/IN them/PPO the/AT Atlanta/NP and/CC Fulton/NP-tl County/NN-tl purchasing/VBG departments/NNS which/WDT it/PPS said/VBD ``/`` ARE/BER well/QL operated/VBN and/CC follow/VB generally/RB accepted/VBN practices/NNS which/WDT inure/VB to/IN the/AT best/JJT interest/NN of/IN both/ABX governments/NNS ''/'' ./.
```
Different datasets might have different forms, but each word (token) in sentences will be generally accompanied by a POS tag to form the pair **word/tag**.

Let's explore the [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus) and the [Penn Treebank Corpus](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.8216&rep=rep1&type=pdf) annotated with POS tags.
Both corpora are already included in NLTK with POS tags, and we can get the full text by calling `tagged_words()` or `tagged_sents()` when the corpus is also segmented into sentences.

In [ ]:
# read Brown Corpus
print(nltk.corpus.brown.tagged_words())
print(nltk.corpus.brown.tagged_sents())

In [ ]:
# read Penn Treebank Corpus
print(nltk.corpus.treebank.tagged_words())
print(nltk.corpus.treebank.tagged_sents())

Did you notice that the tags used in the two corpora seem different? They are indeed using different tagsets, and to investigate what each tag category means, we can check both tagsets: 

In [ ]:
# Brown tagset
nltk.help.brown_tagset()

In [ ]:
# Penn Treebank tagset
nltk.help.upenn_tagset()

Taking the Brown Corpus as an example, we can explore the corpus further by counting how many times a tag appears in the corpus, and create a distribution of tags.

In [ ]:
tag_fd = nltk.FreqDist(tag for (word, tag) in nltk.corpus.brown.tagged_words())
tag_fd

We can find the most and least frequent POS tag by sorting the dictionary.

In [ ]:
tag_list = sorted(tag_fd.items(), key=lambda kv: kv[1], reverse=True)
print(tag_list[0])
print(tag_list[-1])

We can also plot the curve of the POS tag frequency distribution. For the simplicity of visualization, let's first map each tag name to an index.

In [ ]:
tag_fd_index = dict(zip([tag for tag, freq in tag_list], range(len(tag_list))))

Then let's create a new `FreqDict` of tag_index and its frequency.

In [ ]:
tag_index_fd = nltk.FreqDist({tag_fd_index[k]: v for k, v in tag_fd.items()})
tag_index_fd.plot()

Finally, let’s look for words that are highly ambiguous as to their POS tag. Understanding why such words are tagged as they are in each context can help us clarify the distinctions between the tags.

Note that the items being counted in the frequency distribution are word-tag pairs, we can then treat the word as a condition and the tag as an event, and initialize a conditional frequency distribution with a list of condition-event
pairs. This lets us see a frequency-ordered list of tags given a word:

In [ ]:
data = nltk.ConditionalFreqDist((word.lower(), tag) for (word, tag) in nltk.corpus.brown.tagged_words())
print(data.conditions()[0])
data['the']

We can print first 20 words along with their POS tags when the word has more than 4 tags.

In [ ]:
idx = 0
for word in data.conditions():
    if len(data[word]) > 4:
        tags = list(data[word].keys())
        print (word, ' '.join(tags))
        idx += 1
        if idx >= 20:
            break

> **<h3>💻 Try it yourself!</h3>**

For Penn Treebank Corpus, repeat the same process, i.e. plotting the distribution of the tagset, and finding the most and least frequent tag.

For both corpora, what kinds of words occur in the noun, adjective, adverb category?

Explore the POS ditribution of ambiguous words mentioned before, e.g. *content* and *present*. Do they support our observations?

#### POS tag n-grams
So far we have been working with a only single POS tag in text sequence, i.e. the unigram. 
However, we should be aware that POS each tag in text sequence often depends on its previous/later tags.
For example, nouns can appear after determiners and adjectives, and can be the subject or object of the verb.
Therefore, we can look at some n-gram sequences of POS tags such as tag bi-grams.

Let's build a frequency dictionary of tag bi-grams for the Brown Corpus.

In [ ]:
tag_bigram_list = list(nltk.bigrams(nltk.corpus.brown.tagged_words()))
tag_bigram_fd = nltk.FreqDist((a[1], b[1]) for (a, b) in tag_bigram_list if b[1])
tag_bigram_fd

We can see that compared to unigram tag frequency dictionary, bigram dictionary has many more entries. It can potentiall cause the data sparsity problmen as we will encounter later.

To validate our intuitions that nouns often occur after the determiners and adjectives, we can investigate the distribution of tag bi-gram **(X, NN)** where **X** refers to any possible tag.

In [ ]:
nn_bigram_fd = nltk.FreqDist({k: v for k, v in tag_bigram_fd.items() if k[1] == 'NN'})
sorted_nn_bigram_list = sorted(nn_bigram_fd.items(), key=lambda kv: kv[1], reverse=True)
print(sorted_nn_bigram_list[:20])

We can see that the two most frequent tags with **(X, NN)** are indeed **AT** (article) and **JJ** (adjective) according to the Brown Corpus tagset.

### Mapping of properties using Python dictionaries
As we have seen, a tagged word of the form **(word, tag)** is an association between a word and a POS tag. 
Once we start doing POS tagging, we will be using such structures extensively.
For example, we want to get the tag list of a given word **(word, tag list)**, or the frequency of a given tag **(tag, freq)**.
We can think of this process as mapping from a certain property to another, and the most natural way to store mappings in Python uses the so-called `dictionary` data type, which we have encountered in module 1 and previous part of this module. 

We look at dictionaries and see how they can represent a variety of language information. 
For example, recall the frequency dictionary we used previously, which maps from a POS tag to its frequency.

In [ ]:
print(tag_fd['NN'])
tag_fd

If we want to create a dictionary of the form **(word, its corresponding tags)**, what types would be for the key and value of the dictionary?

In [ ]:
word2tags = {'present': 'V', 'object': 'N'}
for k, v in word2tags.items():
    print('{}: {}'.format(k, word2tags[k]))

Here we use the `string` type for tag, which can be problematic, as both words *present* and *object* can be either noun and verb, so if we want to change a tag of *present*, the original tag will be overwritten.

In [ ]:
word = 'present'
print('{}: {}'.format(word, word2tags[word]))
print(word2tags['present'])
word2tags['present'] = 'N'
print('{}: {}'.format(word, word2tags[word]))

A better way is to use the mapping **(word (string), tags (list))** for the dictionary.

In [ ]:
word2tags = {'present': ['V', 'N'], 'object': ['V', 'N']}
for k, v in word2tags.items():
    print('{}: {}'.format(k, word2tags[k]))

And if we want to further add a new tag to the word, we can simply `append` it to the tag list.

In [ ]:
word = 'present'
print('{}: {}'.format(word, word2tags[word]))
word2tags[word].append('ADJ')
print('{}: {}'.format(word, word2tags[word]))

#### Default dictionaries

If we try to access a key that is not in a dictionary, we get an error. 
However, it’s often useful if a dictionary can automatically create an entry for this new key and give it a default value, such as zero or the empty list.
Both **NLTK** and **Python** provide us with the `defaultdict`, a more advanced dictionary, where we can supply a parameter which can be used to create the default value, e.g., int, float, str, list, dict, tuple.
For simplicity, we will stick to the `defaultdict` of NLTK, and for that of Python you can refer to the [link](https://docs.python.org/3/library/collections.html#collections.defaultdict).

Here we assign 0 frequency to a new tag *UNK*, and a tag list with the most frequent tag **N** inside it to a new word.

In [ ]:
tag_fd_def = nltk.defaultdict(int, tag_fd)
print(tag_fd_def['UNK'])

In [ ]:
word2tags_def = nltk.defaultdict(lambda: ['N'], word2tags)
print(word2tags_def['a_new_word'])

> **<h3>💻 Try it yourself!</h3>**

1. In NLP, one of the preprocessing steps is to replace low-frequency words with a special "out of vocabulary" token, *UNK*, with the help of a default dictionary. 
Preprocess `alice` corpus: keep the most frequent *n* words, and map the rest words to UNK.

In [ ]:
alice = nltk.corpus.gutenberg.words('carroll-alice.txt')

2. Create a POS unigram for the Penn Treebank Corpus of the mapping **(word, tag_list)**.

#### POS tag n-grams for tagging
We can use default dictionaries with complex keys and values to build tag n-grams for tagging.

Taking bi-gram as an example, we want to store the frequency of tag **t2**, which is dependent on the tag **t1** of its previous word, and the current word **w2**.

In [ ]:
# create a nested default dictionary, where the value is another default dictionary whose default value is int (0)
pos = nltk.defaultdict(lambda: nltk.defaultdict(int))
# create bigrams of (word, tag) pair
for ((w1, t1), (w2, t2)) in nltk.bigrams(nltk.corpus.treebank.tagged_words()):
    pos[(t1, w2)][t2] += 1
print(pos[('DT', 'right')])

Each time through the loop we updated our `pos` entry for **(t1, w2)**, a tag and its following word. 
When we look up an item in pos we must specify a compound key , and we get back a dictionary object. 
A POS tagger could use such information to decide that the word *right*, when preceded by a determiner, should be tagged as one of the tags in *NN*, *JJ*, and *RB*.

### Automatic Tagging

We will explore various ways to automatically perform POS tagging. We will see that the tag of a word depends on the word and its context within a *sentence*. 
For this reason, we will be working with data at the level of (tagged) sentences rather than words. 

We’ll begin by loading the data we will be using.

In [ ]:
penn_tagged_sents = nltk.corpus.treebank.tagged_sents()
penn_sents = nltk.corpus.treebank.sents()

#### Data Split

As we mentioned earlier, for most machine learning POS taggers, a split of the original dataset into the *training* set and *test* set is necessary.
As Training and testing a model on a single dataset can easily lead to the problem of [overfitting](https://en.wikipedia.org/wiki/Overfitting), as a tagger that simply memorized its training data and made no attempt to construct a general model would get a perfect score, but would be useless for tagging new text. 
Therefore, another held-out test set is used after training the model with training set to evaluate the generalizability of the model.

In this section, we split the data, training on 90% and testing on the remaining 10%, and this proportion may vary depending on the task and data size.

In [ ]:
train_prop = 0.9
size = int(len(penn_tagged_sents) * train_prop)
train_tagged_sents = penn_tagged_sents[:size]
test_tagged_sents = penn_tagged_sents[size:]
train_sents = penn_sents[:size]
test_sents = penn_sents[size:]
print('Training sents: {} {}'.format(len(train_sents), len(train_tagged_sents)))
print('Test sents: {} {}'.format(len(test_sents), len(test_tagged_sents)))

Now we split the original annotated dataset into a training and test set. we will evaluate ALL taggers on the test set, and use training data for taggers that require data to train on.

> **<h3>💻 Try it yourself!</h3>**

1. Create the tag distribution plot of both *training* and *test* set of Penn Treebank Corpus, and compare them with the plot of the whole corpus. What difference can you see?
2. Measure the out-of-vocabulary (OOV) rate between the training and test sets:
    - report the number of tokens and token types in the 90% training set;
    - report the number of tokens and token types in the 10% test set;
    - report the TEST token OOV rate and token type OOV rate, i.e. how many of test that are not present in training. 

#### Evaluation

Evaluation is another core part of NLP.
When evaluating our models on test set annotated by human experts, what numbers are we going to report? An intuitive one is the *accuracy* comparing the gold standard test set tags and the predicted tags produced by the model, i.e. the rate of correct tags the model has predicted.

Recall the term *presicion*, *recall* and *F1-score* for 2-class classification in previous modules, it is still applicable in POS tagging, where we can have these number for EACH POS tag category (NN v.s. non NN). We can achieve this by building a *confusion matrix* of the outputs.

Let's use the tagger in the very beginning of the section and create such confusion matrix:


In [ ]:
gold_test_tags = [tag for test_tagged_sent in test_tagged_sents for (word, tag) in test_tagged_sent]
pred_test_tags = [tag for test_sent in test_sents for (word, tag) in nltk.pos_tag(test_sent)]
print(nltk.ConfusionMatrix(gold_test_tags, pred_test_tags).pretty_format())

As we can see, the row values indicate the POS tags given by reference (gold standard test set), and the column tags are from the model output.

In order to calculate the metrics we mentioned earlier, let's first read the whole matrix in:

In [ ]:
import numpy as np
cm = nltk.ConfusionMatrix(gold_tags, test_tags)
cm_matrix = np.array(cm._confusion)
print(cm_matrix)

The total number of tags are just the sum of the matrix value.

In [ ]:
total_tags = np.sum(cm_matrix)
total_tags

*Accuracy* is simply the ratial between the correct tags (sum of the diagnoal value called *trace*) and total tag number.

In [ ]:
acc = np.trace(cm_matrix) / total_tags
acc

Taking **NN** as an example, we want to calualte the *precision*, *recall* and *F1-score*.
First we need to calculate true positive, false positive and false negative.
True positive is simply the entry where both row and column are **NN**.

In [ ]:
nn_idx = cm._indices['NN']
tp = cm_matrix[nn_idx][nn_idx]
tp

False positive is the sum of *NN* column values exluding true positive. 

In [ ]:
fp = sum(cm_matrix[:, nn_idx]) - tp
fp

Similarly, False negative is the sum of *NN* row values exluding true positive.

In [ ]:
fn = sum(cm_matrix[nn_idx, :]) - tp
fn

The precision, recall and F1-score can be caclculated accordingly:

In [ ]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
print(precision, recall, f1)

> **<h3>💻 Try it yourself!</h3>**

1. Calculate the three metrics for each POS category with the same tagger, and complete the table below.

| POS           | Precision     | Recall  | F1-score 
| ------------- |---------------|---------|----------
| NN      | 0.86 | 0.93 | 0.89 
| $       |      |      |      
| ''      |      |      |      
| Add tag here   |      |      |      

#### Default tagger
The simplest possible tagger assigns the same tag to each token. This may seem to be a rather banal step, but it establishes an important baseline for tagger performance. 
In order to get the best result, we tag each word with the most likely tag.

In [ ]:
nltk.FreqDist(gold_test_tags).max()

Now we can create a tagger that tags everything as **NN**.

In [ ]:
default_tagger = nltk.DefaultTagger('NN')
default_preds = [(word, tag) for test_sent in test_sents for (word, tag) in default_tagger.tag(test_sent)]
default_preds[:20]

In [ ]:
The accuracy of this tagger is

In [ ]:
default_pred_tags = [tag for (word, tag) in default_preds]
default_cm = nltk.ConfusionMatrix(gold_tags, default_pred_tags)
default_cm_matrix = np.array(default_cm._confusion)
acc = np.trace(default_cm_matrix) / np.sum(default_cm_matrix)
acc

Alternatively, we can directly calculate accuracy with `evaluate()`.

In [ ]:
acc_sys = default_tagger.evaluate(test_tagged_sents)
acc_sys

> **<h3>💻 Try it yourself!</h3>**

1. Calculate the three metrics for each POS category with the **default** tagger, and complete the table below.

| POS           | Precision     | Recall  | F1-score 
| ------------- |---------------|---------|----------
| NN      |  |  |  
| $       |      |      |      
| ''      |      |      |      
| Add tag here   |      |      |      

#### The most frequent 100 unigram tagger
A lot of high-frequency words do not have the **NN** tag. 
Let’s find the 100 most frequent words and store their most likely tag. We can then use this information as the
model for a *lookup tagger*.

START FROM HERE

In [ ]:
# create a FreqDist of word unigram from training corpus
fd = nltk.FreqDist([word for train_tagged_sent in train_tagged_sents for (word, tag) in train_tagged_sent])
# get the 100 most frequent unigram word 
most_freq_words = list(fd.keys())[:100]
# create a ConditionalFreqDist associating the word and POS tags
cfd = nltk.ConditionalFreqDist([(word, tag) for train_tagged_sent in train_tagged_sents for (word, tag) in train_tagged_sent])
# get the most likely tag for each word
likely_tags = dict((word, cfd[word].max()) for word in most_freq_words)
baseline_tagger = nltk.UnigramTagger(model=likely_tags)
baseline_tagger.evaluate(test_tagged_sents)

** The most frequent 100 unigram tagger
Again, evaluate it using recall, precision and F1 for each category 
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
Again encourage the students to look and learn from the evaluation data
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
** A simple bi-gram tagger
Emphasize that a real n-gram tagger would do decoding using a dynamic programming algorithm called Viterbi (the students will have heard of this in the lecture).  Today in the interests of time we are not going to do decoding, we’re just going to assign the most frequent POS sequence to each bigram that matches (if indeed a bigram does match).
Again, evaluate it using recall, precision and F1 for each category 
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
Low accuracy because of data sparsity – most bigrams were never seen during training.
Extension task: get the students to test their tagger on the GENIA corpus. What is the accuracy? What does this tell us about the tagger, about the training data (representativeness) and how could the tagger be improved?
-	Get the students to report F1 scores for each category and for all categories in a LaTeX table (template is provided).
** Combining a models with backoff (e.g. unigram plus bigram tagger)
** Further insights from evaluation using a confusion matrix

In [ ]:
print('hello world!')

In [ ]:
int(3.5)

In [ ]:
len([1,2,3])

Another similar concept is `method`. Check [here](https://stackoverflow.com/questions/155609/whats-the-difference-between-a-method-and-a-function) for their difference.

Now let's learn how to define and call functions.

### Define a function

A function is defined by using the `def` keyword, followed by 
1. a **function name** of your choosing;
2. a set of parentheses which hold any **parameters** the function will take (they can be empty);
3. an ending **colon**;
4. function **content code**.

Let's define a simple function `hello()`that prints **hello world**.

In [ ]:
def hello():
    print('hello world')

Note that 
- We do not have any parameter for this function, and we will discuss on it in more details later;
- To start our real function content code, python requires a 4-space indent, as shown above.

Now we have defined our first function. In order to let the program run this function, we still need to call the function, just like what we did to the python built-in functions:

In [ ]:
hello()

Functions can be more complicated. For example, we can use `for` loops, conditional statements, and more within our function block.

For example, the function defined below utilizes a conditional statement to check if the input for the name variable contains a vowel, then uses a for loop to iterate over the letters in the name string.

In [ ]:
# Define function names()
def names():
    # Set up name variable with input
    name = str(input('Enter your name: '))
    # Check whether name has a vowel
    if set('aeiou').intersection(name.lower()):
        print('Your name contains a vowel.')
    else:
        print('Your name does not contain a vowel.')

    # Iterate over name
    for letter in name:
        print(letter)

In [ ]:
# Call the function
names()

The `names()` function sets up a conditional statement and a for loop, showing how code can be organized within a function definition.

> Defining functions within a program makes our code modular and reusable so that we can call the same functions without rewriting them.

### Function parameters

So far we have looked at functions with empty parentheses, but we can define parameters in function definitions within their parentheses.

> A parameter is a variable in the definition of a function that the function can accept.

Let’s create a function that takes trhee parameters `x`, `y`, `z`, and adds them in different configurations. The sums of these will be printed by the function. Then we’ll call the function and pass numbers into the function.

In [ ]:
def add_numbers(x, y, z):
    a = x + y
    b = x + z
    c = y + z
    print(a, b, c)

Now we can pass the `arguments` we want into the function to call it. Check [here](https://www.quora.com/What-is-the-difference-between-argument-and-parameters-in-C) for the difference between parameter and argument.

In [ ]:
add_numbers(1, 2, 3)

We passed the number `1` in for the `x` parameter, `2` in for the `y` parameter, and `3` in for the `z` parameter. These values correspond with each parameter in the order they are given.

In [ ]:
# try different arguments
add_numbers(4, 6, 8)
add_numbers('a', 'b', 'c') # how does it work?

<p style="font-size:1.5em; font-weight: bold">
<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/exercise.png?raw=1" style="height:36pt; display:inline; vertical-align:bottom; margin-right: 4pt" /> 
Try it yourself! <hr>
</p>

Write a function that, given an input list of integers, prints their average.

<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><code>
  def get_average_list(l):
      print( sum(l) / len(l) )
  </code></p>
</details> 

<p style="font-size:1.5em; font-weight: bold">
<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/exercise.png?raw=1" style="height:36pt; display:inline; vertical-align:bottom; margin-right: 4pt" /> 
Try it yourself! <hr>
</p>

Write a function that, given an input string, computes and prints the following statistics:
- Total number of characters.
- Total number of tokens.
- Total number of word types (i.e., distinct tokens) *Hint: there is a datatype that can help you with this!*

In [ ]:
def string_statistics(s):
    tot_characters = #TODO
    tot_tokens = #TODO
    tot_types = #TODO
    
    print("The given input string contains", tot_characters, "characters,",\
         tot_tokens, "tokens and", tot_types, "types")

# Test your function here!
test_string = "To be or not to be, not to be or to be!"
string_statistics(test_string) 

### Keyword Arguments

We can also use keyword arguments in a function call to identify the arguments by the parameter name.

When using keyword arguments, we can use parameters out of order because the Python interpreter will use the keywords provided to match the values to the parameters:

In [ ]:
add_numbers(x = 4, y = 6, z = 8)
add_numbers(y = 6, x = 4, z = 8) # out of order, but same arguments

### Default Argument Values

We can also provide default values for one or more parameters, so the parameters will be set to the default values
if we do not mention them when calling the function.

In [ ]:
def add_numbers_default_value(x, y = 6, z = 10):
    a = x + y
    b = x + z
    c = y + z
    print(a, b, c)

In [ ]:
add_numbers_default_value(4)
add_numbers_default_value(x = 4)

We can also explicitly pass the arguments to the parameters with the default value:

In [ ]:
add_numbers_default_value(4, y = 8)

### Returning a Value

We can pass parameters into a function, and a function can also return a value to us in the end. When a function exits, it can *optionally* pass an expression back to the caller, with the `return` statement. If you use a `return` statement with no arguments, the function will return `None`.

So far, we have used the `print()` statement instead of the return statement in our functions. Let’s create a program that instead of printing will return a variable. The function `square` will take a parameter `x`, and returns the variable `y` representing the square of `x`.

In [ ]:
def square(x):
    y = x ** 2
    return y

In [ ]:
result = square(3)
print(result)

As stated previously, we can use `return` with no arguments, so that the function will return `None`.

In [ ]:
# we return no arguments
def square_return_noarg(x):
    y = x ** 2
    return

In [ ]:
result = square_return_noarg(3)
print(result)

Additionally, without using the `return` statement here, the function cannot return a value so the value defaults to `None` as well.

In [ ]:
# we do no use return
def square_noreturn(x):
    y = x ** 2

In [ ]:
result = square_noreturn(3)
print(result)

In the previous function `add_numbers`, instead of printing the results, we can `return` more than one value wrapped in data structures such as **tuples**, **lists** or **dictionaries**:

In [ ]:
def add_numbers_return_dict(x, y, z):
    a = x + y
    b = x + z
    c = y + z
    return {'a': a, 'b': b, 'c': c}

In [ ]:
result = add_numbers_return_dict(1, 2, 3)
print(result)

Whenever the program hits a `return` statement, the function will exit immediately, whether or not they are returning a value.

In [ ]:
def loop_five():
    for x in range(0, 25):
        print(x)
        if x == 5:
            # Stop function at x == 5
            return
    print("This line will not execute.")

loop_five()

The function hits `return` before the `for` loop ends, so the line that is outside of the loop will not run. 

<p style="font-size:1.5em; font-weight: bold">
<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/exercise.png?raw=1" style="height:36pt; display:inline; vertical-align:bottom; margin-right: 4pt" /> 
Try it yourself! <hr>
</p>

Write a function that:
- Takes as input a shopping list in the form of `items : quantity` pairs;
- For each item to buy, prints a reminder in the form: *Don't forget to buy* + `quantity` + `items`;
- Ends by wishing you a nice shopping time.

In [ ]:
def print_shopping_list(shopping_list):
    # write here your function
    pass

# Test your function here!
shopping_list = {"pears": 5, "apples": 7, "bananas": 4}
print_shopping_list(shopping_list)

<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><code>
    def print_shopping_list(shopping_list):
        for item, quantity in shopping_list.items():
            print("Don't forget to buy", quantity, item)
        print("Have a nice shopping time!")
  </code></p>
</details> 

Now, let's make our function a bit nicer.
What would happen, for example, if someone would enter an incorrect input?
Our function would crash!

Let's improve our `print_shopping_list` function in the cell above in this way:
- At the beginning of the function, check that the given input is of type `dict`;
- If the input doesn't pass this test, return a `print` statement that asks the user for a correct input.

Now, test our improved funtion with the test inputs below.

In [ ]:
# Test your improved function here!
print_shopping_list({"pears": 5, "apples": 7, "bananas": 4})
print_shopping_list(["pears", "apples", "bananas"])
print_shopping_list({"tomatoes": 9, "cat_litter": 1, "thuna_chunks": 4})

<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><code>
    def print_shopping_list(shopping_list):
        if type(shopping_list) != dict:
            return("Please, give me a valid shopping list!")
        for item, quantity in shopping_list.items():
            print("Don't forget to buy", quantity, item)
        print("Have a nice shopping time!")
  </code></p>
</details> 

<p style="font-size:1.5em; font-weight: bold">
<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/exercise.png?raw=1" style="height:36pt; display:inline; vertical-align:bottom; margin-right: 4pt" /> 
Try it yourself! <hr>
</p>

Now, let's write a function that returns the total cost of a shopping list given as inputs two dictionaries: a `shopping_list` dictionary composed of `item : quantity` pairs (as in the excercise above) and an `inventory` dictionary composed of `item : cost` pairs.


Complete the function below with the missing statements.

In [ ]:
def calculate_shopping_bill(shopping_list, inventory):
    shopping_cost = 0
    for item, quantity in shopping_list.items():
        # TODO: get the total price by summing up the prices of single items

    return shopping_cost
        
    
# Test your function here!
our_shopping = {"pears": 5, "apples": 7, "bananas": 4}
current_inventory = {"pears": 0.5, "apples": 0.7, "bananas": 1.3}

cost = calculate_shopping_bill(our_shopping, current_inventory)
print("You'll spend", cost, "GBP")

<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><code>
    def calculate_shopping_bill(shopping_list, inventory):
        shopping_cost = 0
        for item, quantity in shopping_list.items():
            single_item_cost = inventory[item]
            total_item_cost = single_item_cost * quantity
            shopping_cost += total_item_cost
        return shopping_cost
  </code></p>
</details> 

<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/quiz.png?raw=1" style="height:36pt; float:left; margin-right: 4pt" /> <h2>Quiz</h2>

Given the function:
```
def add_numbers(x = 3, y = 4, z = 5):
    x = x + y
    y = x + z
    z = y + z
    print(x, y, z)
    
o, p, q = 5, 6, 7
L = [5,6,7]
```

1. What does the following code print?

```
d = add_numbers(5)
```

A. 7, 8, 9

B. 7, 8, 9

C. 9, 14, 19

D. 9, 14, 19

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>C</p>
</details> 

2. What is the value of `d`?

A. 7

B. 9

C. 10

D. None

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>D</p>
</details> 

3. What is the value of `o p q` after running the following code?

```
add_numbers(o, p, q)
```

A. 7, 12, 17

B. 9, 14, 19

C. 11, 18, 25

D. 5, 6, 7

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>D</p>
</details> 

4. How to pass the element of `L` as the parameters of the function?


A. ```add_numbers(L)```


B. ```add_numbers(L*)```

C. ```add_numbers(*L)```

D. ```add_numbers(**L)```

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>C</p>
</details> 

5. What is the value of `L` after running the following code?

```
add_numbers(L, L, L)
```

A. [5, 6, 7]

B. [5, 6, 7, 5, 6, 7]

C. [5, 6, 7, 5, 6, 7, 5, 6, 7]

D. [5, 6, 7, 5, 6, 7, 5, 6, 7, 5, 6, 7]

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>A</p>
</details> 

<h1 align='center' style='margin-top:2em'>
    <img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/section_header.png?raw=1" 
         style="height:36pt; display:inline; vertical-align:center; margin-top:0em" />
    <u>Regular Expressions</u>
    <img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/section_header.png?raw=1" 
         style="height:36pt; display:inline; vertical-align:center; margin-top:0em" />
<hr>
</h1>

This section is heavily built on this [tutorial](https://docs.python.org/3.6/howto/regex.html#regex-howto), and students are strongly recommended to go through it. A more comprehensive documentation of python regular expressions can be found [here](https://docs.python.org/3.6/library/re.html).

Regular expressions (called REs, or regexes, or regex patterns) are essentially a tiny, highly specialized programming language embedded inside Python and made available through the re module. Using this little language, you specify the rules for the set of possible strings that you want to match; this set might contain English sentences, or e-mail addresses, or TeX commands, or anything you like. You can then ask questions such as “Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”. You can also use REs to modify a string or to split it apart in various ways.

Python uses the **raw string** notation for RE patterns, and backslashes `'\'` are not handled in any special way in a string literal prefixed with `r`. So `r'\n'` is a two-character string containing `'\'` and `'n'`, while `'\n'` is a one-character string containing a newline. 

We’ll use the raw string notation for the rest of the section. We will also write REs in highlight style , i.e. `r'\n'` is equivalent to `\n`, usually without quotes, and strings to be matched 'in single quotes'.

REs can contain both special and ordinary characters. Most ordinary characters, like `A`, `a`, or `0`, are the simplest REs; they simply match themselves. You can concatenate ordinary characters to match more complex sequence. For example, `test` will match the string 'test' exactly.

Some characters are special metacharacters, and don’t match themselves. Instead, they signal that some out-of-the-ordinary thing should be matched, or they affect other portions of the RE by repeating them or changing their meaning.

Here’s a complete list of the metacharacters and their explanations:

- `.`             Matches any character except a newline.
- `^`             Matches the start of the string.
- `$`             Matches the end of the string or just before the newline at the end of the string.
- `*`             Matches 0 or more (greedy) repetitions of the preceding RE. Greedy means that it will match as many repetitions as possible.
- `+`             Matches 1 or more (greedy) repetitions of the preceding RE.
- `?`             Matches 0 or 1 (greedy) of the preceding RE.
- `*?, +?, ??`    Non-greedy versions of the previous three special characters.
- `{m,n}`         Matches from m to n repetitions of the preceding RE.
- `{m,n}?`        Non-greedy version of the above.
- `\\`            Either escapes special characters or signals a special sequence.
- `[]`            Indicates a set of characters. A "^" as the first character indicates a complementing set.
- `|`             A|B, creates an RE that will match either A or B.
- `(...)`         Matches the RE inside the parentheses. The contents can be retrieved or matched later in the string.


The special sequences consist of "\\" and a character from the list below:
- `\number`  Matches the contents of the group of the same number.
- `\A`       Matches only at the start of the string.
- `\Z`       Matches only at the end of the string.
- `\b`       Matches the empty string, but only at the start or end of a word.
- `\B`       Matches the empty string, but not at the start or end of a word.
- `\d`       Matches any decimal digit; equivalent to the set `[0-9]`.
- `\D`       Matches any non-digit character; equivalent to `[^\d]`.
- `\s`       Matches any whitespace character; equivalent to `[ \t\n\r\f\v]`.
- `\S`       Matches any non-whitespace character; equivalent to `[^\s]`.
- `\w`       Matches any alphanumeric character; equivalent to `[a-zA-Z0-9_]`.
- `\W`       Matches the complement of `\w`.
- `\\`       Matches a literal backslash.

In [ ]:
# import built-in regular expression module in python
import re

In [ ]:
text = ("""                2
  When forty winters shall besiege thy brow,"""
  "And dig deep trenches in thy beauty's field, "
  "Thy youth's proud livery so gazed on now, "
  "Will be a tattered weed of small worth held: "
  "Then being asked, where all thy beauty lies, "
  "Where all the treasure of thy lusty days; "
  "To say within thine own deep sunken eyes, "
  "Were an all-eating shame, and thriftless praise. "
  "How much more praise deserved thy beauty's use, "
  "If thou couldst answer 'This fair child of mine "
  "Shall sum my count, and make my old excuse' "
  "Proving his beauty by succession thine. "
  "This were to be new made when thou art old, "
  "And see thy blood warm when thou feel'st it cold.")

print(text)

We will mainly concentrate two essential usage of regular expressions: **searching** and **substituting** text.

### Searching text

Let's first find word '**where**', in regardless of case.

The **re** module provides an interface to the regular expression engine, allowing you to compile REs into objects and then perform matches with them.

In [ ]:
p = re.compile(r'\b[Ww]here\b')
p

After we obtained compiled pattern object, it has several functions and attributes. For **searching** text, there are mainly four functions available:
- *match()*: determine if the RE matches at the beginning of the string.
- *search()*: scan through a string, looking for any location where this RE matches.
- *findall()*: find all substrings where the RE matches, and returns them as a list.
- *finditer()*: find all substrings where the RE matches, and returns them as an iterator.

Please consult the **re** documentation for a complete listing.

In [ ]:
m = p.match(text)
print(m)

In [ ]:
m = p.search(text)
print(m)

The functions *match()* and *search()* return None if no match can be found. If they’re successful, a match object instance is returned, containing information about the match: where it starts and ends, the substring it matched, and more.

In this example, as the text doesn't start with the word 'where', so *match()* won't find any match, whereas *search()* will look for any location where RE matches.

We can further query the match object for information about the matching string.

In [ ]:
m.group()

In [ ]:
m.start(), m.end()

In [ ]:
m.span()

Now what if we want to find all matches, i.e. all words 'where'? We can use *findall()* and *finditer()*.

The *findall()* returns a list of matching strings.

In [ ]:
m = p.findall(text)
print(m)

The *finditer()* returns a sequence of match object instances as an iterator.

In [ ]:
iterator = p.finditer(text)
for match in iterator:
    print(match)

You don’t have to create a pattern object and call its functions; the **re** module also provides the same top-level functions.

In [ ]:
print(re.search(r'\b[Ww]here\b', text))

In [ ]:
print(re.findall(r'\b[Ww]here\b', text))

Notice that when we compile the RE, there is a **re.UNICODE** item when we print the object. It is a compliation flag which let you modify some aspects of how RE works. The full flag list is available in the module documentation. For example, we can use **re.IGNORECASE** or **re.I** to do case-insensitive matches, so that we do not have to take care of capital letters.

In [ ]:
p = re.compile(r'\bwhere\b', re.I)
p

And we will still have the same results.

In [ ]:
m = p.findall(text)
print(m)

Now try to come up with your regular expressions and search in the text to see if they will work.

You can use the cell below:

#### Groupings

Frequently you need to obtain more information than just whether the RE matched or not. REs are often used to dissect strings by writing a RE divided into several subgroups which match different components of interest.

Groups are marked by the `'('`, `')'` metacharacters. They group together the expressions contained inside them, and you can repeat the contents of a group with a repeating qualifier, such as `*`, `+`, `?`, or `{m,n}`.

In [ ]:
p = re.compile('((a(b)c)d)*')
m = p.match('abcdabcd')
print(m.span())

Groups indicated with `'('`, `')'` also capture the starting and ending index of the text that they match; this can be retrieved by passing an argument to `group()`, `start()`, `end()`, and `span()`. 

Groups are numbered starting with 0. Group 0 is always present; it’s the whole RE, so previous methods all have group 0 as their default argument.

In [ ]:
print(m.start(0), m.end(0))
print(m.group(0,1,2,3))
print(m.span(2))

The `groups()` method returns a tuple containing the strings for all the subgroups, from 1 up to however many there are.

In [ ]:
print(m.groups())

### Substituting text

We will use the following functions for substitution:
- *sub()*: find all substrings where the RE matches, and replace them with a different string.
- *subn()*: does the same thing as sub(), but returns the new string and the number of replacements.

Again, you are encouraged to refer to the docs for more functions.

***sub*** *(replacement, string[, count=0])*

Returns the string obtained by replacing the leftmost non-overlapping occurrences of the RE in *string* by the replacement *replacement*. If the pattern isn’t found, string is returned unchanged.

The optional argument *count* is the maximum number of pattern occurrences to be replaced; *count* must be a non-negative integer. The default value of 0 means to replace all occurrences.

For the same example, let's substitute all `'where'` words to `'-------'`.

In [ ]:
p = re.compile(r'\b[Ww]here\b')
print(p.sub('------', text))

In [ ]:
print(p.sub('------', text, count = 1))

The `subn()` method does the same work, but returns a 2-tuple containing the new string value and the number of replacements that were performed:

In [ ]:
print(p.subn('------', text))

In [ ]:
p = re.compile('section{ (?P<name> [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\g<1>}','section{First} section{second}')

<p style="font-size:1.5em; font-weight: bold">
<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/exercise.png?raw=1" style="height:36pt; display:inline; vertical-align:bottom; margin-right: 4pt" /> 
Try it yourself! <hr>
</p>

Now it's your turn! Please write a regular expression to match sentences that start with the word “This” (case insensitive).

And a regex that matches words which are emoticons (like *:)*). At least 5 emoticons must be matched.

And a regex that can match words which contains at least 3 *o*s in a row (so that *cooooool* would be a match, but *book* would not).

Finally, write a regex that substitutes any word-ending in *a* or *as* into an *o* or *os*.

Test your regex by converting the Spanish sentence *Maria es ecuatoriana y tiene dos hijas rubias y muy altas* (Mary is Ecuadorian and has two blonde and very tall daughters) into its masculine equivalent.

### Building a simple Eliza

Having learnt about Eliza in the lecture, which is meant to emulate a Rogerian psychologist, let us now build a simple version of Eliza with RE.

Firstl, let us build a dictionary `grefs` that is used to map first-person pronouns to second-person pronouns and vice-versa. It is used to “reflect” a statement back against the user:

In [ ]:
grefs = {
    "am":          "are",
    "was":         "were",
    "i":           "you",
    "i'd":         "you would",
    "i've":        "you have",
    "i'll":        "you will",
    "my":          "your",
    "are":         "am",
    "you are":     "I am",
    "you've":      "I have",
    "you'll":      "I will",
    "your":        "my",
    "yours":       "mine",
    "you":         "me",
    "me":          "you"
}

Then we need another table `gpats` that is made up of a list of lists, where the first element is a RE that matches the user’s statements and the second element is a list of potential responses. 

Many of the potential responses contain placeholders that can be filled in with fragments to echo the user’s statements.

In [ ]:
gpats = [
  [r'I need (.*)',
  [  "Why do you need {0}?",
    "Would it really help you to get {0}?",
    "Are you sure you need {0}?"]],

  [r'I can\'?t (.*)',
  [  "How do you know you can't {0}?",
    "Perhaps you could {0} if you tried.",
    "What would it take for you to {0}?"]],

  [r'I am (.*)',
  [  "Did you come to me because you are {0}?",
    "How long have you been {0}?",
    "How do you feel about being {0}?"]],

  [r'I\'?m (.*)',
  [  "How does being {0} make you feel?",
    "Do you enjoy being {0}?",
    "Why do you tell me you're {0}?",
    "Why do you think you're {0}?"]],

  [r'Are you ([^\?]*)\??',
  [  "Why does it matter whether I am {0}?",
    "Would you prefer it if I were not {0}?",
    "Perhaps you believe I am {0}.",
    "I may be {0} -- what do you think?"]],

  [r'What (.*)',
  [  "Why do you ask?",
    "How would an answer to that help you?",
    "What do you think?"]],

  [r'How (.*)',
  [  "How do you suppose?",
    "Perhaps you can answer your own question.",
    "What is it you're really asking?"]],

  [r'Because (.*)',
  [  "Is that the real reason?",
    "What other reasons come to mind?",
    "Does that reason apply to anything else?",
    "If {0}, what else must be true?"]],

  [r'Hello(.*)',
  [  "Hello... I'm glad you could drop by today.",
    "Hi there... how are you today?",
    "Hello, how are you feeling today?"]],

  [r'quit',
  [  "Thank you for talking with me.",
    "Good-bye.",
    "Thank you, that will be $150.  Have a good day!"]],
  
  [r'(.*)',
  [  "Please tell me more.",
    "Let's change focus a bit... Tell me about your family.",
    "Can you elaborate on that?",
    "Why do you say that {0}?",
    "I see.",
    "Very interesting.",
    "{0}.",
    "I see.  And what does that tell you?",
    "How does that make you feel?",
    "How do you feel when you say that?"]] 
]

Now let us write the function `respond` that reads the user input and generates the response accordingly.

We iterate through the regular expressions in `gpats`, trying to match each one with the user’s input `s`. If we find a match, we choose a response template randomly from the list of possible responses associated with the matching pattern. Then we interpolate the match groups from the RE into the response string, calling the `transalte` function on each match group first.

When we use the list comprehension to generate a list of reflected match groups, we explode the list with the asterisk `\*` character before passing it to the string’s `format` method. Format expects a series of positional arguments corresponding to the number of format placeholders – `{0}`, `{1}`, etc. – in the string.

In [ ]:
import re
import random
def respond(s):
    for pattern, responses in gpats:
        # find a match for s
        match = re.match(pattern, s)
        if match:
            # chosen randomly from among the available options
            response = random.choice(responses)
            return response.format(*[translate(g) for g in match.groups()])

First, we make the input lowercase, then we tokenize it by splitting on whitespace characters. We iterate through the list of tokens and, if the token exists in our reflections dictionary, we replace it with the value from the dictionary. So “I” becomes “you”, “your” becomes “my”, etc.

In [ ]:
def translate(fragment):
    tokens = fragment.lower().split()
    for i, token in enumerate(tokens):
        if token in grefs:
            tokens[i] = grefs[token]
    return ' '.join(tokens)

Finally, we can wrap everything into an interface where our Eliza will wait for the user input and generate corresponding response.

In [ ]:
def start_eliza():
    print('Therapist\n---------')
    print('Talk to the program by typing in plain English, using normal upper-')
    print('and lower-case letters and punctuation.  Enter "quit" when done.')
    print('='*72)
    print('Hello.  How are you feeling today?')

    s = ''
    while s != 'quit':
        try:
            s = input('> ')
            while s[-1] in '!.':
                s = s[:-1]
        except:
            s = 'quit'
            print('Invalid Input, exit ...')
        print(respond(s))

In [ ]:
start_eliza()

Now try to add your own [RE, responses] pairs to the `gpats` to capture more questions from the user. Feel free to refer to [this](https://github.com/jezhiggins/eliza.py/blob/master/eliza.py) for more REs.

<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/quiz.png?raw=1" style="height:36pt; float:left; margin-right: 4pt" /> <h2>Quiz</h2>

1. For the previous text, which RE can find all words with 3, 4 and 5 characters?

A. `r'.{3,5}'`

B. `r'\b.{3,5}+\b'`

C. `r'\b\w{3,5}\b'`

D. `r'\w{3,5}'`

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>C</p>
</details> 

2. Which function can you use to find all the words that the RE in 1. matches?

A. `match()`

B. `search()`

C. `finditer()`

D. `findall()`

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>C, D</p>
</details> 

3. How many words the RE in 1 mathes are there?

A. 128

B. 105

C. 96

D. 77

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>D</p>
</details> 

4. How many words are there with 5 letters?

A. 10

B. 20

C. 30

D. 35

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>B</p>
</details> 

5. What's the span of the first word with 4 letters?

A. (16, 20)

B. (18, 22)

C. (20, 24)

D. (22, 26) 

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>C</p>
</details> 

<h1 align='center' style='margin-top:2em'>
    <img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/section_header.png?raw=1" 
         style="height:36pt; display:inline; vertical-align:center; margin-top:0em" />
    <u>SpaCy</u>
    <img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/section_header.png?raw=1" 
         style="height:36pt; display:inline; vertical-align:center; margin-top:0em" />
<hr>
</h1>

spaCy is a free, open-source library for advanced Natural Language Processing (NLP) in Python.

spaCy is designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning.

There are many useful resources online for spaCy, among which the official [tutorial](https://spacy.io/usage/spacy-101) and the [course](https://course.spacy.io/) are extremely useful.

### Installation

Follow the link [here](https://spacy.io/usage) to install spaCy with pip. Then download and install the English model by running:
```
python -m spacy download en_core_web_sm
```

spaCy is enables many different features introduced [here](https://spacy.io/usage/spacy-101#features), and we will focus on the simplest and direct features:
- tokenization;
- stemming; (not supported by spaCy)
- lemmatization;
- part-of-speech (POS) tagging;
- sentence boundary detection and sentence segmentation;
- dependency parsing;

In [ ]:
# import spacy module
import spacy

# load english model
nlp = spacy.load("en_core_web_sm")
# read the text and process
doc = nlp(text)

In [ ]:
# Tokenization
# get all separated tokens in doc
tokens = [token.text for token in doc]
print(tokens)

You can see from the list *tokens* that there each tokenized token is in the entry of the list.

In [ ]:
# Lemmatization
lemmas = [token.lemma_ for token in doc]
print(lemmas)

Did you notice that almost all pronouns are subsituted with `-PRON-`?

Unlike verbs and common nouns, there’s no clear base form of a personal pronoun. Should the lemma of “me” be “I”, or should we normalize person as well, giving “it” — or maybe “he”? spaCy’s solution is to introduce a novel symbol, `-PRON-`, which is used as the lemma for all personal pronouns.

In [ ]:
# POS tagging
poss = [token.pos_ for token in doc]
print(poss)

Compare the outputs of the previous three lists.

Now let's move to sentence level features, i.e. sentence boundary detection and sentence segmentation. Unlike other libraries, spaCy uses the dependency parse to determine sentence boundaries. This is usually more accurate than a rule-based approach, but it also means you’ll need a statistical model and accurate predictions.
If your texts are closer to general-purpose news or web text, this should work well out-of-the-box. 

In [ ]:
sentences = list(doc.sents)
sent_texts = [sentence.text for sentence in sentences]
print(sent_texts)
print(len(sent_texts))

However, for other texts like social media texts, your application may benefit from a custom rule-based implementation. You can either use the built-in Sentencizer or plug an entirely custom rule-based function into your processing pipeline.

Let's build a rule-based sentence segmentor.

In [ ]:
from spacy.lang.en import English

nlp_rule = English()  # just the language with no model
sentencizer = nlp_rule.create_pipe("sentencizer")
nlp_rule.add_pipe(sentencizer)
doc_rule = nlp_rule(text)
sent_rule_texts = [sent.text for sent in doc_rule.sents]
print(sent_rule_texts)
print(len(sent_rule_texts))

You can see clearly the difference in both methods, where the statistical dependency parse sentence segmentor generate 12 sentencs, but the rule-based sentence segmentor only yields 4 sentences.

Finally, let's have a look at the dependencies, i.e. the dependency relations between tokens, which is also used to segment sentences for the first model.

In [ ]:
# Dependencies
deps = [token.dep_ for token in doc]
print(deps)

Using spaCy’s built-in [displaCy visualizer](https://spacy.io/usage/visualizers), here’s what our example sentence and its dependencies.

In [ ]:
from spacy import displacy
displacy.render(sentences, style = "dep")

<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/quiz.png?raw=1" style="height:36pt; float:left; margin-right: 4pt" /> <h2>Quiz</h2>

<img src="https://github.com/cambridgeltl/python4cl/blob/module_2.3/resources/assessment.png?raw=1" style="height:36pt; float:left; margin-right: 4pt" /> <h2>Assessment</h2>

1. Write a function to calculate the $N_{th}$ Finbonacci number without `for` loop using the folloing formula:
$$
f_n = \frac{a^n - b^n}{a - b} = \frac{a^n - b^n}{\sqrt{5}}
$$
where $a = \frac{1 + \sqrt{5}}{2}$ and $b = \frac{1- \sqrt{5}}{2}$.

In [ ]:
import math
SQRT_5 = math.sqrt(5)
def calc_fibonacci(n):
    a = (1 + SQRT_5) / 2
    b = (1 - SQRT_5) / 2
    fn = int((a**n - b**n) / SQRT_5)
    return fn

print(calc_fibonacci(5))
print(calc_fibonacci(10))
print(calc_fibonacci(20))

2. Tokenize the following sentences with RE:
```
sents = """He asked: "When will you graduate?"\n"I will get my Ph.D. degree(Doctor of Philosophy) in a few years, hehe. And get $123,45675.45." I answered. END"""
```
How many token types are there in total?

In [ ]:
import re

PUNCTS = ['"', "'",':', ';', '?', '(',')', '[', ']', '!', '$', '&', '*', '#', '-']
PUNCTS_REGEX =  {',':['[^ ](,)(\s+)'], '.':['[^ ]\.(\s+)([^a-z])']}

def my_tokenizer(sents):
    print('Original...')
    print(sents)

    for punt in PUNCTS:
        sents = sents.replace(punt, (' ' + punt + ' '))

    for punct, regs in PUNCTS_REGEX.items():
        for reg in regs:
            p = re.compile(reg)
            m = p.search(sents)
            while m:
                sents = sents[:m.start() + 1] + ' ' + punct + ' ' + sents[m.start() + 2:]
                m = p.search(sents)
    sents = re.sub(r' +', ' ', sents)        

    print('After Tokenization...')
    print(sents)
    return sents

sents = """He asked: "When will you graduate?"\n"I will get my Ph.D. degree(Doctor of Philosophy) in a few years, hehe. And get $123,45675.45." I answered. END"""
tok_sents = my_tokenizer(sents)
print(set(tok_sents.split()))
print(len(set(tok_sents.split())))

3. Tokenize the previous sentences with SpaCy. How many token types are there in total?

In [ ]:
# import spacy module
import spacy

# load english model
nlp = spacy.load("en_core_web_sm")
# read the text and process
doc = nlp(sents)
print(doc)
tokens = [token.text for token in doc]
print(' '.join(tokens))
print(set(tokens))
print(len(set(tokens)))

## Links

- [How To Define Functions in Python 3](https://www.digitalocean.com/community/tutorials/how-to-define-functions-in-python-3)
- [Regular Expression HOWTO](https://docs.python.org/3.6/howto/regex.html#regex-howto)
- [Regular Expression Documentation](https://docs.python.org/3.6/library/re.html)
- [Eliza](https://github.com/jezhiggins/eliza.py/blob/master/eliza.py)
- [spaCy 101: Everything you need to know](https://spacy.io/usage/spacy-101#_title)
- [Advanced NLP with spaCy](https://course.spacy.io/)

# Next Module

[Click here](../module_1.4/module_1.4.ipynb) to move to the next module.